In [1]:
%load_ext autoreload
%autoreload 2

### trading features

within each entered trade, i should have:

- enter_price,rolling_price, sl, tp, strat, enter_dfname, enter_time, current_price, highs,lows,hl_pairs

first strat:

- rolling_price=enter_price
- if current_price < ((1+sl)*rolling_price ):
    - exit
- elif current_price > ((1+tp)*rolling_price ):
    - exit
- elif current_price > ((1+tp*0.9)*rolling_price ):
    - sl = max(sl,tp*0.8)
    - tp = tp*1.5 # possible to optimise this with STD instead
- elif current_price > ((1+tp*0.5)*rolling_price ):
    - sl = max(sl,tp*0.2)
    - tp = tp*1.5

hold

In [61]:
def strat_rollingtpsl1(enter_data,strat_data,cur_price):# only HOLD or SELL 
    if cur_price < ((1+strat_data["cur_sl"])*enter_data["price"] ):
        return "SELL",strat_data,f"Stop L{strat_data['cur_sl']*100:.2f}%"
    elif cur_price > ((1+strat_data["cur_tp"])*enter_data["price"]  ):
        return "SELL",strat_data,f"Take P{strat_data['cur_tp']*100:.2f}%"
    # strats to shrink tp and sl
    elif cur_price > ((1+strat_data["cur_tp"]*0.9)*enter_data["price"]  ):
        strat_data["cur_sl"] = max(strat_data["cur_sl"],strat_data["cur_tp"]*0.8+strat_data["slip"])
        strat_data["cur_tp"] = strat_data["cur_tp"]*1.5
        return "HOLD",strat_data,f"UpFast L{strat_data['cur_sl']*100:.2f}% P{strat_data['cur_tp']*100:.2f}%"
    elif cur_price > ((1+strat_data["cur_tp"]*0.5)*enter_data["price"]  ):
        strat_data["cur_sl"] = max(strat_data["cur_sl"],strat_data["cur_tp"]*0.2+strat_data["slip"])
        strat_data["cur_tp"] = strat_data["cur_tp"]*1.4
        return "HOLD",strat_data,f"UpMedm L{strat_data['cur_sl']*100:.2f}% P{strat_data['cur_tp']*100:.2f}%"
    elif cur_price > ((1+strat_data["cur_tp"]*0.3)*enter_data["price"]  ):
        strat_data["cur_sl"] = max(strat_data["cur_sl"],strat_data["cur_tp"]*0.1+strat_data["slip"])
        strat_data["cur_tp"] = strat_data["cur_tp"]*1.3
        return "HOLD",strat_data,f"UpSlow L{strat_data['cur_sl']*100:.2f}% P{strat_data['cur_tp']*100:.2f}%"
    return "HOLD",strat_data,f"In SLTP L{strat_data['cur_sl']*100:.2f}% P{strat_data['cur_tp']*100:.2f}%"

In [15]:
cur_price=100
sl=-0.01
tp=0.1
dfname=""
ent_time=""
hl_pairs=""
enter_data = {"price":cur_price,"sl":sl,"tp":tp,"dfname":dfname,"ent_time":ent_time,
              "hl_pairs":hl_pairs,"strat":"strat_tpsl1"}
strat_data = {"cur_sl":sl,"cur_tp":tp,"slip":-0.002,"prev_price":cur_price,"ent_sl":sl,"ent_tp":tp,"strat":"strat_rollingtpsl1"}
strat_data = {"cur_sl":sl,"cur_tp":tp,"slip":-0.002,"ent_sl":sl,"ent_tp":tp,"strat":"strat_tpsl1"}
# sl tp are in fractions, they are initalised once
# cur_sl cur_tp are in fractions, they are relative to enter price
# "slip" is the margin of error to account for slippage
def strat_tpsl1(enter_data,strat_data,cur_price):# only HOLD or SELL
    enter_price = enter_data["price"]
    if cur_price < ((1+strat_data["cur_sl"])*enter_price ): 
        return "SELL",strat_data,f"Stop L{strat_data['cur_sl']*100:.2f}%"
    elif cur_price > ((1+strat_data["cur_tp"])*enter_price ): 
        return "SELL",strat_data,f"Take P{strat_data['cur_tp']*100:.2f}%"
    # strats to shrink tp and sl
    width=enter_data["tp"]-enter_data["sl"]
    #print("width",width)
    strat_status="In SLTP"
    if cur_price > ( (1+strat_data["cur_sl"]+width*0.9)*enter_price):
        strat_data["cur_sl"] = strat_data["cur_sl"]+width*0.8
        strat_data["cur_tp"] = strat_data["cur_tp"]+width*0.8
        strat_status = "UpFast"
    elif cur_price > ( (1+strat_data["cur_sl"]+width*0.6)*enter_price ):
        strat_data["cur_sl"] = strat_data["cur_sl"]+width*0.2
        strat_data["cur_tp"] = strat_data["cur_tp"]+width*0.5
        strat_status = "UpMedi"
    elif cur_price > ( (1+strat_data["cur_sl"]+width*0.3)*enter_price ):
        strat_data["cur_sl"] = strat_data["cur_sl"]+width*0.1
        strat_data["cur_tp"] = strat_data["cur_tp"]+width*0.1
        strat_status = "UpSlow"
    str1=f"LP `{enter_price*(1+strat_data['cur_sl']):.4f}`,`{enter_price*(1+strat_data['cur_tp']):.4f}`"
    str2=f"(`{strat_data['cur_sl']*100:.2f}`,`{strat_data['cur_tp']*100:.2f}`)%"
    str3=f"\nNext levels: `{(1+strat_data['cur_sl']+width*0.3)*enter_price:.4f}`,"\
         f"`{(1+strat_data['cur_sl']+width*0.6)*enter_price:.4f}`,"\
         f"`{(1+strat_data['cur_sl']+width*0.9)*enter_price:.4f}`"
    str4=f"(`{(strat_data['cur_sl']+width*0.3)*100:.2f}`,"\
         f"`{(strat_data['cur_sl']+width*0.6)*100:.2f}`,"\
         f"`{(strat_data['cur_sl']+width*0.9)*100:.2f}`)%"
    str5=f"width={width:.4f}"
    return "HOLD",strat_data,f"{strat_status} {str1} {str2}{str3}{str4}, {str5}"
def price_action_signal(enter_data,strat_data,cur_price):
    if enter_data["strat"]=="strat_rollingtpsl1":
        return strat_rollingtpsl1(enter_data,strat_data,cur_price)
    elif enter_data["strat"]=="strat_tpsl1":
        return strat_tpsl1(enter_data,strat_data,cur_price)
    else:
        raise NotImplementedError("todo strat :(")

# validate strat

In [16]:
cur_price=100; sl=-0.01; tp=0.05; dfname="";ent_time="";hl_pairs=""
enter_data = {"price":cur_price,"sl":sl,"tp":tp,"dfname":dfname,"ent_time":ent_time,
              "hl_pairs":hl_pairs,"strat":"strat_tpsl1"}
strat_data = {"cur_sl":sl,"cur_tp":tp,"slip":-0.002,"ent_sl":sl,"ent_tp":tp,"strat":"strat_tpsl1"}

In [17]:
print( "start cur_price",cur_price)
print("*"*8)
for cur_price in [105,109,101]:
    print( "cur_price",cur_price)
    data = price_action_signal(enter_data,strat_data,cur_price)
    print(data)
    print(f"cur_price {cur_price:.4f}")
    if data[0]=="HOLD":
        print(f"SL {enter_data['price']*(1+data[1]['cur_sl']):.4f}, TP {enter_data['price']*(1+data[1]['cur_tp']):.4f}") 
    strat_data = data[1]
    print("*"*8)

start cur_price 100
********
cur_price 105
('HOLD', {'cur_sl': 0.038000000000000006, 'cur_tp': 0.098, 'slip': -0.002, 'ent_sl': -0.01, 'ent_tp': 0.05, 'strat': 'strat_tpsl1'}, 'UpFast LP `103.8000`,`109.8000` (`3.80`,`9.80`)%\nNext levels: `105.6000`,`107.4000`,`109.2000`(`5.60`,`7.40`,`9.20`)%, width=0.0600')
cur_price 105.0000
SL 103.8000, TP 109.8000
********
cur_price 109
('HOLD', {'cur_sl': 0.05000000000000001, 'cur_tp': 0.128, 'slip': -0.002, 'ent_sl': -0.01, 'ent_tp': 0.05, 'strat': 'strat_tpsl1'}, 'UpMedi LP `105.0000`,`112.8000` (`5.00`,`12.80`)%\nNext levels: `106.8000`,`108.6000`,`110.4000`(`6.80`,`8.60`,`10.40`)%, width=0.0600')
cur_price 109.0000
SL 105.0000, TP 112.8000
********
cur_price 101
('SELL', {'cur_sl': 0.05000000000000001, 'cur_tp': 0.128, 'slip': -0.002, 'ent_sl': -0.01, 'ent_tp': 0.05, 'strat': 'strat_tpsl1'}, 'Stop L5.00%')
cur_price 101.0000
********


In [21]:
from trader import price_action_signal as pas

In [27]:
cur_price=100; sl=-0.01; tp=0.05; dfname="";ent_time="";hl_pairs=""
enter_data = {"price":cur_price,"sl":sl,"tp":tp,"dfname":dfname,"ent_time":ent_time,
              "hl_pairs":hl_pairs,"strat":"strat_tpsl1"}
strat_data = {"cur_sl":sl,"cur_tp":tp,"slip":-0.002,"ent_sl":sl,"ent_tp":tp,"strat":"strat_tpsl1"}
print( "start cur_price",cur_price)
print("*"*8)
for cur_price in [105,109,101]:
    print( "cur_price",cur_price)
    data = pas(enter_data,strat_data,cur_price)
    print(data[:2])
    print(data[2])
    print(f"cur_price {cur_price:.4f}")
    if data[0]=="HOLD":
        print(f"SL {enter_data['price']*(1+data[1]['cur_sl']):.4f}, TP {enter_data['price']*(1+data[1]['cur_tp']):.4f}") 
    strat_data = data[1]
    print("*"*8)

start cur_price 100
********
cur_price 105
('HOLD', {'cur_sl': 0.038000000000000006, 'cur_tp': 0.098, 'slip': -0.002, 'ent_sl': -0.01, 'ent_tp': 0.05, 'strat': 'strat_tpsl1'})
UpFast LP `103.8000`,`109.8000` (`3.80`,`9.80`)%
Next levels: `105.6000`,`107.4000`,`109.2000`(`5.60`,`7.40`,`9.20`)%, width=0.0600
cur_price 105.0000
SL 103.8000, TP 109.8000
********
cur_price 109
('HOLD', {'cur_sl': 0.05000000000000001, 'cur_tp': 0.128, 'slip': -0.002, 'ent_sl': -0.01, 'ent_tp': 0.05, 'strat': 'strat_tpsl1'})
UpMedi LP `105.0000`,`112.8000` (`5.00`,`12.80`)%
Next levels: `106.8000`,`108.6000`,`110.4000`(`6.80`,`8.60`,`10.40`)%, width=0.0600
cur_price 109.0000
SL 105.0000, TP 112.8000
********
cur_price 101
('SELL', {'cur_sl': 0.05000000000000001, 'cur_tp': 0.128, 'slip': -0.002, 'ent_sl': -0.01, 'ent_tp': 0.05, 'strat': 'strat_tpsl1'})
Stop L5.00%
cur_price 101.0000
********


In [28]:
cur_price=100; sl=-0.01; tp=0.05; dfname="";ent_time="";hl_pairs=""
enter_data = {"price":cur_price,"sl":sl,"tp":tp,"dfname":dfname,"ent_time":ent_time,
              "hl_pairs":hl_pairs,"strat":"strat_tpsl1"}
strat_data = {"cur_sl":sl,"cur_tp":tp,"slip":-0.002,"ent_sl":sl,"ent_tp":tp,"strat":"strat_tpsl1"}
print( "start cur_price",cur_price)
print("*"*8)
for cur_price in [105,109,122]:
    print( "cur_price",cur_price)
    data = pas(enter_data,strat_data,cur_price)
    print(data[:2])
    print(data[2])
    print(f"cur_price {cur_price:.4f}")
    if data[0]=="HOLD":
        print(f"SL {enter_data['price']*(1+data[1]['cur_sl']):.4f}, TP {enter_data['price']*(1+data[1]['cur_tp']):.4f}") 
    strat_data = data[1]
    print("*"*8)

start cur_price 100
********
cur_price 105
('HOLD', {'cur_sl': 0.038000000000000006, 'cur_tp': 0.098, 'slip': -0.002, 'ent_sl': -0.01, 'ent_tp': 0.05, 'strat': 'strat_tpsl1'})
UpFast LP `103.8000`,`109.8000` (`3.80`,`9.80`)%
Next levels: `105.6000`,`107.4000`,`109.2000`(`5.60`,`7.40`,`9.20`)%, width=0.0600
cur_price 105.0000
SL 103.8000, TP 109.8000
********
cur_price 109
('HOLD', {'cur_sl': 0.05000000000000001, 'cur_tp': 0.128, 'slip': -0.002, 'ent_sl': -0.01, 'ent_tp': 0.05, 'strat': 'strat_tpsl1'})
UpMedi LP `105.0000`,`112.8000` (`5.00`,`12.80`)%
Next levels: `106.8000`,`108.6000`,`110.4000`(`6.80`,`8.60`,`10.40`)%, width=0.0600
cur_price 109.0000
SL 105.0000, TP 112.8000
********
cur_price 122
('SELL', {'cur_sl': 0.05000000000000001, 'cur_tp': 0.128, 'slip': -0.002, 'ent_sl': -0.01, 'ent_tp': 0.05, 'strat': 'strat_tpsl1'})
Take P12.80%
cur_price 122.0000
********


In [29]:
cur_price=100; sl=-0.01; tp=0.05; dfname="";ent_time="";hl_pairs=""
enter_data = {"price":cur_price,"sl":sl,"tp":tp,"dfname":dfname,"ent_time":ent_time,
              "hl_pairs":hl_pairs,"strat":"strat_tpsl1"}
strat_data = {"cur_sl":sl,"cur_tp":tp,"slip":-0.002,"ent_sl":sl,"ent_tp":tp,"strat":"strat_tpsl1"}
print( "start cur_price",cur_price)
print("*"*8)
for cur_price in [98.99]:
    print( "cur_price",cur_price)
    data = pas(enter_data,strat_data,cur_price)
    print(data[:2])
    print(data[2])
    print(f"cur_price {cur_price:.4f}")
    if data[0]=="HOLD":
        print(f"SL {enter_data['price']*(1+data[1]['cur_sl']):.4f}, TP {enter_data['price']*(1+data[1]['cur_tp']):.4f}") 
    strat_data = data[1]
    print("*"*8)

start cur_price 100
********
cur_price 98.99
('SELL', {'cur_sl': -0.01, 'cur_tp': 0.05, 'slip': -0.002, 'ent_sl': -0.01, 'ent_tp': 0.05, 'strat': 'strat_tpsl1'})
Stop L-1.00%
cur_price 98.9900
********


# testing popen

In [1]:
import json

In [2]:
trade_params = json.load(open("trade_params.json","r"))
trade_params["ver1"]["params"][0]

{'symbol': 'ETCUSDT', 'interval': '5m', 'sl': -0.1, 'tp': 0.1}

In [3]:
int("0")

0

In [4]:
import trader

In [9]:
%%time
val = trader.get_current_price("ETCUSDT")

CPU times: total: 0 ns
Wall time: 91.1 ms


In [12]:
import subprocess
import time

proc = subprocess.Popen('python test_task.py', shell=True)
print("called")
#time.sleep(3)

called


In [12]:
import time

In [13]:
time.sleep(2)

In [16]:
!python test_runtask2.py

called
sleep10 sec
done sleep


# writing status files

In [10]:
import datetime
str(datetime.datetime.now()).replace(" ","_")

'2023-07-10_17:38:20.304221'

In [5]:
from funcs import get_data
ticker,interval="ETCUSDT","30m"
dfmpl = get_data(ticker,interval,limit=500,type="data")

In [7]:
str(dfmpl.iloc[4].name).replace(" ","_")

'2023-04-24_15:30:00'

In [41]:
signal="ENTER"
closeprice=12.133
dfname=dfmpl.iloc[4].name
with open(f"trade_signals/{ticker}{interval}.status","a") as f:
    f.writelines(f"{signal},{closeprice},{dfname},{datetime.datetime.now()}\n")

In [13]:
with open(f"trade_signals/{ticker}{interval}.status","r") as f:
    signal_raw = f.readlines()
signal=signal_raw[-1].split(",")[0]

In [14]:
signal

'ENTER'

# test binance api

In [2]:
from trader import market_trade
from trader import get_current_price

In [4]:
20/get_current_price("LTCUSDT")

0.2132423499306962

In [2]:
%%time
a1,a2,a3 = market_trade("LTCUSDT",0.15,buy=True,test=False)

CPU times: total: 15.6 ms
Wall time: 118 ms


In [3]:
a1,a2,a3

('FILLED',
 [{'price': '93.20000000',
   'qty': '0.15000000',
   'commission': '0.00015000',
   'commissionAsset': 'LTC',
   'tradeId': 1143099}],
 {'symbol': 'LTCTUSD',
  'orderId': 101055581,
  'orderListId': -1,
  'clientOrderId': 'hxDCFy7Mvh5XsrURYCYOWu',
  'transactTime': 1688979930654,
  'price': '0.00000000',
  'origQty': '0.15000000',
  'executedQty': '0.15000000',
  'cummulativeQuoteQty': '13.98000000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'BUY',
  'workingTime': 1688979930654,
  'fills': [{'price': '93.20000000',
    'qty': '0.15000000',
    'commission': '0.00015000',
    'commissionAsset': 'LTC',
    'tradeId': 1143099}],
  'selfTradePreventionMode': 'NONE'})

In [11]:
0.0001500*100

0.015

In [1]:
from disc_api import ping,STATUS_PING2,SIGNALROLE,CRYPTO_SIGNALS2

In [5]:
ping(STATUS_PING2,"hello")